# What We Want
We want to manipulate the JSON files and make them so the values for heatmap and move paths show the % difference compared to other players.

In [87]:
import pandas as pd
import json
from pandas import compat
from pprint import pprint

## Proof of concept - Grab all the raw moves data from one player

In [88]:
with open('../../app/data/json/as_color/Adams.asblack_stats.json') as data_file:    
    data = json.load(data_file)

In [61]:
moves = data['moves']

In [66]:
moves_df = pd.DataFrame(moves)

In [77]:
moves_df['Bc1'].sum()

4555.0

In [ ]:
moves_df['Bc1'].sum()

In [73]:
moves_percentage = (moves_df/moves_df.sum())*100

### Save as json, dropping Nan values

In [99]:
moves_percentage.to_json('./test_move_percentage.json')

In [93]:
moves_percentage.apply(lambda x: [x.dropna()], axis=1).to_json('./test_move_percentage.json')

# Load in every player's data to get total average

In [236]:
all_gms = ['Adams',
    'Akobian','Akopian','Alburt','Alekhine','Alekseev','Almasi','Anand','Anderssen','Andersson','Andreikin','Aronian','Ashley',
'Averbakh','Azmaiparashvili','Bacrot','Bareev','BecerraRivero','Beliavsky','Benjamin','Benko',
           #'Berliner',
           'Bernstein','Bird','Bisguier','Blackburne',
'Blatny','Bogoljubow','Boleslavsky','Bologan','Botvinnik','Breyer','Bronstein','Browne','Bruzon','Bu','Byrne','Capablanca','Carlsen','Caruana',
'Chiburdanidze','Chigorin','Christiansen','DeFirmian',
           #'DeLaBourdonnais',
           'Denker','Ding','DominguezPerez','Dreev','Dzindzichashvili','Ehlvest',
'Eljanov','Euwe','Evans','Fedorowicz','Fine','Finegold','Fischer','Fishbein','Flohr','Gaprindashvili','Gashimov','Gelfand','Geller','Georgiev','Giri',
'Gligoric','Goldin','GrandaZuniga','Grischuk','Gulko','Gunsberg','GurevichD','GurevichM','Harikrishna','Hort','Horwitz','Hou','Huebner','Ibragimov',
'IllescasCordoba','Inarkiev','Ivanchuk','IvanovA','IvanovI','Ivkov','Jakovenko','Janowski','Jobava','Jussupow','Kaidanov','Kamsky','Karjakin',
'Karpov','Kasimdzhanov','Kasparov','Kavalek','Keres','Khalifman','Kholmov','Koneru','Korchnoi','Korobov','Kosteniuk','Kotov','Kramnik',
'Krasenkow','Krush','Kudrin','Lahno','Larsen','Lasker','Lautier','Le','Leko','Levenfish','Li','Lilienthal','Ljubojevic','Lputian','MacKenzie','Malakhov',
'Mamedyarov','Maroczy','Marshall',
           #'McDonnell',
           'McShane','Mecking','Mikenas','Miles','Milov','Morozevich','Morphy','Motylev','Movsesian',
'Muzychuk','Najdorf','Najer','Nakamura','Navara','Negi','Nepomniachtchi','Ni','Nielsen','Nikolic','Nimzowitsch','Nisipeanu','Novikov','Nunn',
'Olafsson','Oll','Onischuk','Pachman','Paehtz','Panno','Paulsen','Petrosian',
#'Philidor',
           'Pillsbury','Pilnik','PolgarJ','PolgarS','PolgarZ','Polugaevsky',
'Ponomariov','Portisch','Psakhis','Quinteros','Radjabov','Reshevsky','Reti','Ribli','Rohde','Rubinstein','Rublevsky','Saemisch','Sakaev','Salov',
'Sasikiran','Schlechter','Seirawan','Serper','Shabalov','Shamkovich','Shirov','Short','Shulman','Smirin','Smyslov','So','Sokolov','Soltis','Spassky',
'Speelman','Spielmann','Stahlberg','Staunton','Stefanova','Stein','Steinitz','Suetin','SultanKhan','Sutovsky','Svidler','Szabo','Taimanov','Tal',
'Tarrasch','Tartakower','Teichmann','Timman','Tiviakov','Tkachiev','Tomashevsky','Topalov','TorreRepetto','Uhlmann','Unzicker','Ushenina',
'VachierLagrave','Vaganian','VallejoPons','VanWely','Vitiugov','Volokitin','Waitzkin','Wang','WangH','Wei','Winawer','Wojtaszek','Wojtkiewicz',
'Wolff','Xie','Xu','Ye','Yermolinsky','Yu','Yudasin','Zhu','Zukertort','Zvjaginsev']

In [237]:
def load_player_moves(player):
    filename_location_blackmoves = '../../app/data/json/as_color/{}.asblack_stats.json'.format(player)
    with open(filename_location_blackmoves) as data_file:    
        data = json.load(data_file)
        moves = data['moves']
        moves_df = pd.DataFrame(moves)
        moves_df_black = moves_df.fillna(0)
        moves_df_black.drop(['Bc1', 'Bf1', 'Ke1', 'Nb1', 'Ng1',
           'Qd1', 'Ra1', 'Rh1', 'a2', 'b2', 'c2',
           'd2', 'e2', 'f2', 'g2','h2'], axis=1)
    filename_location_whitemoves = '../../app/data/json/as_color/{}.aswhite_stats.json'.format(player)
    with open(filename_location_whitemoves) as data_file:    
        data = json.load(data_file)
        moves = data['moves']
        moves_df = pd.DataFrame(moves)
        moves_df_white = moves_df.fillna(0)
        moves_df_white.drop(['Bc8', 'Bf8', 'Ke8', 'Nb8', 'Ng8',
           'Qd8', 'Ra8', 'Rh8', 'a7', 'b7',
           'c7', 'd7', 'e7', 'f7', 'g7', 'h7'], axis=1)
        moves_df = pd.concat([moves_df_black, moves_df_white])
    return moves_df

In [238]:
def average_by_col(df):
    df = (df/df.sum())*100
    return df

In [279]:
# Setup Dataframe
combined_moves = pd.DataFrame()
combined_moves_average = pd.DataFrame()

for player in all_gms:
    moves = load_player_moves(player)
    moves_average = average_by_col(moves)
    combined_moves = pd.concat([combined_moves,moves]).groupby(level=0).sum()
    combined_moves_average = pd.concat([combined_moves_average,moves_average]).groupby(level=0).sum()
    #print(player)
    #print(combined_moves.head(1))
combined_moves_average = combined_moves_average / (len(all_gms))

In [243]:
short_list = ['Adams','Akobian']

In [281]:
def movepct_vs_everyone(player):
    moves = load_player_moves(player)
    ave = average_by_col(moves)
    ave_vs_group = ave - combined_moves_average
    ave_vs_group_dict = ave_vs_group.to_dict()
    my_dict = {"moves": ave_vs_group_dict}
    pd.DataFrame(my_dict).to_json('./player_vs_jsons/{}.movepct_vs_group.json'.format(player))

In [282]:
for player in all_gms:
    movepct_vs_everyone(player)